In [1]:
import pandas as pd 
import os
import requests as rq
from bs4 import BeautifulSoup as bs
import re
import textwrap
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from datetime import datetime
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

### Correo

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/114.0.0.0 Safari/537.36"
}

In [3]:
with open("APPI OPENAI.txt") as archivo:
  apikey_openai = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey_openai

In [4]:
distritos_lima = [
    "Ancón",
    "Ate",
    "Barranco",
    "Breña",
    "Carabayllo",
    "Chaclacayo",
    "Chorrillos",
    "Cieneguilla",
    "Comas",
    "El Agustino",
    "Independencia",
    "Jesús María",
    "La Molina",
    "La Victoria",
    "Lince",
    "Los Olivos",
    "Lurigancho",
    "Lurín",
    "Magdalena del Mar",
    "Miraflores",
    "Pachacámac",
    "Pucusana",
    "Pueblo Libre",
    "Puente Piedra",
    "Punta Hermosa",
    "Punta Negra",
    "Rímac",
    "San Bartolo",
    "San Borja",
    "San Isidro",
    "San Juan de Lurigancho",
    "San Juan de Miraflores",
    "San Luis",
    "San Martín de Porres",
    "San Miguel",
    "Santa Anita",
    "Santa María del Mar",
    "Santa Rosa",
    "Santiago de Surco",
    "Surquillo",
    "Villa El Salvador",
    "Villa María del Triunfo"
]


In [5]:
departamentos_peru = [
    "Amazonas",
    "Áncash",
    "Apurímac",
    "Arequipa",
    "Ayacucho",
    "Cajamarca",
    "Callao", 
    "Cusco",
    "Huancavelica",
    "Huánuco",
    "Ica",
    "Junín",
    "La Libertad",
    "Lambayeque",
    "Lima",
    "Loreto",
    "Madre de Dios",
    "Moquegua",
    "Pasco",
    "Piura",
    "Puno",
    "San Martín",
    "Tacna",
    "Tumbes",
    "Ucayali"
]


In [6]:
prompt_pais = ChatPromptTemplate.from_messages([
    ("system", """Tu tarea es identificar el o los países mencionados en una noticia.
Responde únicamente con una lista de nombres de países:
- Si se menciona un solo país, devuélvelo en una lista con un solo elemento.
- Si se mencionan varios países, devuélvelos todos en una lista.
- Si no se menciona ningún país, devuelve una lista vacía.
No des explicaciones ni añadas texto adicional. Solo responde con la lista de países."""),
    ("human", "{input}")
])

In [7]:
llm = ChatOpenAI(
    temperature=0,
    model="gpt-4")

def pais_detector(texto: str) -> list:
    respuesta = llm.invoke(prompt_pais.format_messages(input=texto)).content.strip().lower()
    return respuesta

def buscar_ordenados(texto, lista):
    texto_lower = texto.lower()
    encontrados = []
    for lugar in lista:
        patron = r'\b' + re.escape(lugar.lower()) + r'\b'
        match = re.search(patron, texto_lower)
        if match:
            encontrados.append((lugar, match.start()))  
    encontrados.sort(key=lambda x: x[1])
    return [nombre for nombre, _ in encontrados]

In [8]:
lista_url = []
lista_titulo = []
lista_texto = []
lista_fecha = []
lista_distrito = []
lista_departamento = []
lista_pais = []

In [9]:
for pagina in tqdm(range(1,20)):
    url = f'https://diariocorreo.pe/buscar/mujer/todas/descendiente/{pagina}/'
    try: 
        html = rq.get(url,headers=headers).content.decode("utf-8")
        soup = bs(html) #creamos nuestro arbol
        enlaces_raw = soup.find_all('a',{'itemprop':'url','class':'story-item__title block overflow-hidden primary-font line-h-xs mt-10'})
        links = ['https://diariocorreo.pe'+ link.get('href') for link in enlaces_raw if 'href' in link.attrs ] 
        for noticia in tqdm(links): 
            try:
                soup2 = bs(rq.get(noticia, headers=headers).content.decode("utf-8")) 
                try:
                    titulo = soup2.find_all("h1")[0].text
                    fecha = "/".join(list((re.findall(r"(\d{2})[/-](\d{2})[/-](\d{4})",soup2.find_all('time')[0].text))[0]))
                    body = " ".join([p.text for p in soup2.find_all("div", class_="story-contents__content")[0].find_all('p')])
                    body2 = titulo  + ' ' + body
                    distrito = buscar_ordenados(body2, distritos_lima)
                    departamento = buscar_ordenados(body2, departamentos_peru)
                    pais = pais_detector(body)
                    lista_pais.append(pais)
                    lista_url.append(noticia)
                    lista_titulo.append(titulo)
                    lista_texto.append(body)
                    lista_fecha.append(fecha)
                    lista_departamento.append(departamento)
                    lista_distrito.append(distrito)
                except:
                    print(f'Error al encontrar metadata de la noticia: No se encontro metadata: {noticia}\n')
            except Exception as e:
                print(f"Error al procesar noticia: {noticia}\n{e}")
    except Exception as e:
        print(f"Error al cargar la página {pagina}:\n{e}")

 11%|█         | 2/19 [02:07<17:59, 63.48s/it]

Error al encontrar metadata de la noticia: No se encontro metadata: https://diariocorreo.pe/videos/actualidad/surco-roban-celulares-a-mujer-en-puerta-de-condominio-delincuente-asalto-robo-noticia/



 16%|█▌        | 3/19 [03:14<17:14, 64.69s/it]

Error al encontrar metadata de la noticia: No se encontro metadata: https://diariocorreo.pe/edicion/piura/piura-condenan-a-sujeto-a-14-meses-por-agresion-a-una-mujer-noticia/



 21%|██        | 4/19 [04:23<16:38, 66.60s/it]

...

Error al encontrar metadata de la noticia: No se encontro metadata: https://diariocorreo.pe/videos/actualidad/mujer-pierde-la-vida-tras-ser-estrangulada-por-su-pareja-en-huaura-video-peru-lima-pnp-feminicidio-noticia/



Error al encontrar metadata de la noticia: No se encontro metadata: https://diariocorreo.pe/videos/actualidad/la-libertad-mujer-pierde-la-vida-a-manos-de-su-expareja-mientras-dormia-en-su-vivienda-video-peru-pnp-feminicidio-noticia/



Error al encontrar metadata de la noticia: No se encontro metadata: https://diariocorreo.pe



Error al encontrar metadata de la noticia: No se encontro metadata: https://diariocorreo.pe



100%|██████████| 20/20 [00:38<00:00,  1.95s/it]
0it [00:00, ?it/s]5/19 [16:06<03:53, 58.29s/it]
0it [00:00, ?it/s]6/19 [16:08<02:03, 41.25s/it]
0it [00:00, ?it/s]7/19 [16:10<00:58, 29.40s/it]
0it [00:00, ?it/s]8/19 [16:11<00:20, 21.00s/it]
100%|██████████| 19/19 [16:13<00:00, 51.23s/it]


In [10]:
df = pd.DataFrame({
    'url': lista_url,
    'titulo': lista_titulo,
    'texto': lista_texto,
    'fecha': lista_fecha,
    'distrito': lista_distrito,
    'departamento': lista_departamento,
    'pais': lista_pais})

In [11]:
df

,url,titulo,texto,fecha,distrito,departamento,pais
0,https://diariocorreo.pe/espectaculos/dona-peta...,Doña Peta le llama la atención a sobrino de Pa...,En varias ocasiones se ha comentado sobre el f...,31/05/2025,[],[],"[""perú"", ""argentina""]"
1,https://diariocorreo.pe/edicion/huanuco/huanuc...,Huánuco: mujer es acuchillada por su expareja ...,Ante la negativa de no querer retomar la relac...,30/05/2025,[],[Huánuco],[]
2,https://diariocorreo.pe/edicion/puno/puno-muje...,Puno: Mujer de 83 años muere atropellada cuand...,Una tragedia enlutó nuevamente las carreteras ...,28/05/2025,[],"[Puno, Cusco]","[""perú""]"
3,https://diariocorreo.pe/edicion/puno/puno-dict...,Puno: Dictan prisión preventiva contra mujer i...,El Juzgado de Investigación Preparatoria de C...,27/05/2025,[],[Puno],"[""perú""]"
4,https://diariocorreo.pe/edicion/arequipa/arequ...,Arequipa: Mujer es sospechosa de pepear a admi...,La comerciante Yudy Marlene Valeriano Vilchez ...,26/05/2025,[],[Arequipa],[]
...,...,...,...,...,...,...,...
423,https://diariocorreo.pe/miscelanea/viral-mujer...,Mujer se convierte en viral de TikTok por su r...,La delincuencia es una constante en las ciudad...,29/04/2022,[],[],"[""méxico""]"
424,https://diariocorreo.pe/edicion/lima/expo-muje...,Expo Mujer inicia ciclo de capacitación para M...,Expo Mujer como parte de su Programa de Capac...,15/04/2022,[],[],"[""perú""]"
425,https://diariocorreo.pe/edicion/lima/callao-fi...,Callao: filman a joven mujer encabezando atrac...,Cámaras de seguridad registraron el accionar d...,20/01/2022,[],[Callao],"[""perú""]"
426,https://diariocorreo.pe/edicion/lima/el-increi...,Conoce el caso de la mujer “secuestrada” a la ...,Un giro increíble dieron las investigaciones s...,26/10/2021,"[San Juan de Lurigancho, Lurigancho]",[La Libertad],"[""colombia""]"


In [12]:
df.to_excel('Noticias_correo.xlsx',index=False)